In [14]:
import pandas as pd
import numpy as np
np.int = int
np.bool = bool

from sklearn.metrics import median_absolute_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_absolute_error, max_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from skopt import BayesSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline

In [2]:

scans=pd.read_excel("../dummy_data.xlsx")
print(scans.shape)

(550, 139)


In [3]:
carbon = scans.copy()
carbon['protocol_map'] = carbon['protocol_map'].str.upper()

In [5]:
#clear missing data
carbon['Age at Egg Collection'] = carbon['Age at Egg Collection'].replace(-100, np.nan)
carbon['Age at Egg Collection'] = round(carbon['Age at Egg Collection'],0)
carbon['BMI'] = np.where((carbon['BMI'] < 18) | (carbon['BMI'] > 50), np.nan, carbon['BMI'])
carbon['trigger_map'] = carbon['trigger_map'].replace('NONE', np.nan)
carbon['protocol_map'] = carbon['protocol_map'].replace('NONE', np.nan)
carbon['days to trig'] = carbon['days to trig'].apply(lambda x: x if 5<=x<=30 else np.nan)

In [6]:
mat_scans = carbon[carbon['No. Mature Eggs']>-1]

In [7]:
mat_scans = mat_scans[~(pd.isna(mat_scans['DoT Follicles']))]

print(mat_scans.shape)

(550, 139)


In [9]:

mat_scans = mat_scans[['6_mm', '7_mm', '8_mm', '9_mm','10_mm','11_mm','12_mm','13_mm','14_mm','15_mm','16_mm','17_mm','18_mm','19_mm','20_mm','21_mm','22_mm','23_mm','24_mm','25_mm','26_mm','Age at Egg Collection','BMI', 'days to trig', 'trigger_map', 'protocol_map', 'e2_dot', 'No. Mature Eggs', 'Clinic']].dropna()
print(mat_scans.shape)


(550, 29)


In [10]:
X = mat_scans[['6_mm', '7_mm', '8_mm', '9_mm','10_mm','11_mm','12_mm','13_mm','14_mm','15_mm','16_mm','17_mm','18_mm','19_mm','20_mm','21_mm','22_mm','23_mm','24_mm','25_mm','26_mm','Age at Egg Collection','BMI', 'days to trig', 'trigger_map', 'protocol_map', 'e2_dot']]

y = mat_scans[['No. Mature Eggs']]

y=y.values.ravel()

y = np.log(y+1)

by_clinic = mat_scans['Clinic']
by_clinic = by_clinic.values.ravel()

In [31]:
X.shape

(550, 27)

In [32]:
y.shape

(550,)

In [33]:
by_clinic.shape

(550,)

In [15]:

param_grid_2 = {
    'model__max_iter': (5, 10),
    'model__learning_rate': (0.0001,0.1),
    'model__l2_regularization': (0.0, 1.0),
    'model__min_samples_leaf': (5, 20),
    'model__loss': ('squared_error','absolute_error','poisson')
}

outer_cv = LeaveOneGroupOut()
outer_cv.get_n_splits(X,y,by_clinic)
splits = outer_cv.get_n_splits(groups=by_clinic)

# Initialize arrays to store the results
outer_scores = []
outer_scores_2 = []
best_params_list_2 = pd.DataFrame(index=param_grid_2)
inner_results_2 = []
df_perm_importance_2 = pd.DataFrame(index=X.columns)

y_pred_all = []


In [16]:


categorical_features=['trigger_map', 'protocol_map']


ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan, encoded_missing_value=np.nan),

        make_column_selector(dtype_include=["category", "object"]),
    ),
    remainder="passthrough",
    verbose_feature_names_out=True,
)


In [17]:

# Perform nested cross-validation
for i, (train_index, test_index) in enumerate(outer_cv.split(X, y, by_clinic)):
    # Check the indices
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clinic_train, clinic_test = by_clinic[train_index], by_clinic[test_index]

    print(X_train.shape)
    print(y_train.shape)

    inner_cv = LeaveOneGroupOut()


    #generate encoder based on HHistGradientBoostingRegressor model
    cat_bool=['ordinalencoder__protocol_map', 'ordinalencoder__trigger_map']

    hist_native = Pipeline(steps=[('preproc', ordinal_encoder),('model', HistGradientBoostingRegressor(categorical_features=cat_bool))

    ]).set_output(transform="pandas")

    bayes_opt_2 = BayesSearchCV(
                    estimator=hist_native,
                    search_spaces=param_grid_2,
                    n_iter=30,
                    n_points=5,
                    scoring='neg_mean_absolute_error',
                    cv=inner_cv,
                    n_jobs=-1,
                    refit=True,
                    verbose=1,
                    return_train_score=True
                    )
    bayes_opt_2.fit(X=X_train, y=y_train, groups=clinic_train)

    # Get the best parameters and the corresponding model
    best_params_2 = bayes_opt_2.best_params_
    best_model_2 = bayes_opt_2.best_estimator_
    # Store the best parameters
    best_params_list_2[f'best_params_run_{i}'] = best_params_2
    print(f"best inner score_2{i}:", bayes_opt_2.best_score_)

    inner_results_2.append(pd.DataFrame(bayes_opt_2.cv_results_))

    y_pred = best_model_2.predict(X_test)

    y_pred = np.exp(y_pred) - 1

    y_test = np.exp(y_test) - 1

    mae= mean_absolute_error(y_test, y_pred)
    print(f'mae_{i}', mae)
    r2= r2_score(y_test,y_pred)
    medae = median_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_test,y_pred)
    max_er = max_error(y_test, y_pred)
    outer_scores.append([mae,r2,medae,mape, rmse, max_er])

    perm_result_2 = permutation_importance(best_model_2, X_test, y_test, n_repeats=5, scoring='neg_mean_absolute_error')

     # Append mean and std of each run to the DataFrame for the current outer fold
    df_perm_importance_2[f'mean_run_{i}'] = perm_result_2.importances_mean
    df_perm_importance_2[f'std_run_{i}'] = perm_result_2.importances_std

    y_pred_all.append([y_test,y_pred])


# Calculate average of means across each row
df_perm_importance_2['final_mean'] = df_perm_importance_2.filter(regex=r'^mean_run_\d+$').mean(axis=1)

# Calculate average of standard deviations across each row
df_perm_importance_2['final_sd'] = np.sqrt(df_perm_importance_2.filter(regex=r'^std_run_\d+$').pow(2).mean(axis=1))

# Concatenate the inner_results DataFrames
inner_results_df_2 = pd.concat(inner_results_2)

column_names = ['MAE', 'R2', 'MedAE', 'MAPE', 'RMSE', 'Max Error']
outer_scores_df = pd.DataFrame(outer_scores, columns=column_names)
print("development done")

mae_values = [score[0] for score in outer_scores]
r2_values = [score[1] for score in outer_scores]
medae_values= [score[2] for score in outer_scores]
mape_values= [score[3] for score in outer_scores]
rmse_values= [score[4] for score in outer_scores]
maxerror_values= [score[5] for score in outer_scores]

mae_mean = np.mean(mae_values)
mae_std = np.std(mae_values)

r2_mean = np.mean(r2_values)
r2_std = np.std(r2_values)

medae_mean = np.mean(medae_values)
medae_std = np.std(medae_values)

mape_mean = np.mean(mape_values)
mape_std = np.std(mape_values)

rmse_mean = np.mean(rmse_values)
rmse_std = np.std(rmse_values)

maxerror_mean = np.mean(maxerror_values)
maxerror_std = np.std(maxerror_values)

mae_mean = round(mae_mean, 4)
mae_std = round(mae_std, 4)
r2_mean = round(r2_mean, 4)
r2_std = round(r2_std, 4)

medae_mean = round(medae_mean, 4)
medae_std = round(medae_std, 4)

mape_mean = round(mape_mean, 4)
mape_std = round(mape_std, 4)

rmse_mean = round(rmse_mean, 4)
rmse_std = round(rmse_std, 4)

maxerror_mean = round(maxerror_mean, 4)
maxerror_std = round(maxerror_std, 4)

cv_averaged = {
    'Metric': ['MAE', 'R2', 'MedAE', 'MAPE', 'RMSE', 'Max Error'],
    'Mean': [mae_mean, r2_mean, medae_mean, mape_mean, rmse_mean, maxerror_mean],
    'Standard Deviation': [mae_std, r2_std, medae_std, mape_std, rmse_std, maxerror_std],
}

final_cv_scores = pd.DataFrame(cv_averaged)


print("MAE Mean:", mae_mean)
print("MAE Standard Deviation:", mae_std)
print("R2 Mean:", r2_mean)
print("R2 Standard Deviation:", r2_std)

import os
from datetime import datetime

# Create a folder in the current directory
folder_name = "MII"
os.makedirs(folder_name, exist_ok=True)
model_name = "MII_MULTI_VAR_E2"
# Get the current date in UK date format (DD-MM-YYYY)
current_date = datetime.now().strftime("%d-%m-%Y %H-%M")


file_name = f"{current_date}_HGBR_{model_name}_BestParams2.csv"
csv_file_path = os.path.join(folder_name, file_name)
best_params_list_2.to_csv(csv_file_path)


file_name = f"{current_date}_HGBR_{model_name}_InnerRuns2.csv"
csv_file_path = os.path.join(folder_name, file_name)
inner_results_df_2.to_csv(csv_file_path)

file_name = f"{current_date}_HGBR_{model_name}_Importances2.csv"

csv_file_path = os.path.join(folder_name, file_name)
df_perm_importance_2.to_csv(csv_file_path)


file_name = f"{current_date}_HGBR_{model_name}_OuterScores.csv"
csv_file_path = os.path.join(folder_name, file_name)
outer_scores_df.to_csv(csv_file_path)
print("Files saved to:", folder_name)

file_name = f"{current_date}_HGBR_{model_name}_Metrics.csv"
csv_file_path = os.path.join(folder_name, file_name)
final_cv_scores.to_csv(csv_file_path)

(500, 27)
(500,)
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits


C:\Users\sh2316\OneDrive - Imperial College London\PhD\TFP_Follicle_Sizes\venv\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_20: -0.42573636026375095
mae_0 3.7086725291901286
(500, 27)
(500,)
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits


C:\Users\sh2316\OneDrive - Imperial College London\PhD\TFP_Follicle_Sizes\venv\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sh2316\OneDrive - Imperial College London\PhD\TFP_Follicle_Sizes\venv\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 10 folds for each of 5 candidates, totalling 50 fits


...

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_25: -0.42962135194865486
mae_5 2.4889683046642626
(500, 27)
(500,)
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_26: -0.422283779194483
mae_6 3.053174741619612
(500, 27)
(500,)
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits


C:\Users\sh2316\OneDrive - Imperial College London\PhD\TFP_Follicle_Sizes\venv\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_27: -0.422480733658373
mae_7 2.822182657907248
(500, 27)
(500,)
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_28: -0.4316673999300054
mae_8 2.8722384428510397
(500, 27)
(500,)
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_29: -0.42533521816220876
m

C:\Users\sh2316\OneDrive - Imperial College London\PhD\TFP_Follicle_Sizes\venv\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sh2316\OneDrive - Imperial College London\PhD\TFP_Follicle_Sizes\venv\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 10 folds for each of 5 candidates, totalling 50 fits
best inner score_210: -0.42552386059150604
mae_10 2.5063685208748883
development done
MAE Mean: 2.8967
MAE Standard Deviation: 0.4023
R2 Mean: 0.0915
R2 Standard Deviation: 0.1038
Files saved to: MII
